# Data Collector
In this notebook, we collect the simulation data on qulacs.

## QRAO on regular graphs

In [2]:
# import packages
from maxcut_instance_generator import regular_graph
from encoding import RandomAccessEncoder
from vqe import VQEForQRAO
from rounding import MagicRounding

In [3]:
import os
import pickle
from tqdm.auto import tqdm

# function to run QRAO
def run_qrao(m, n, instance, max_level, root_path, shots):
    qrac = RandomAccessEncoder(m, n)
    hamiltonian = qrac.generate_hamiltonian(instance)
    # print(f"Hamiltonian is {hamiltonian}")
    num_qubit = len(qrac.qubit_to_vertex_map)
    num_edge = len(qrac.calculate_edge_among_qubits(instance))
    # print(f"{num_qubit} qubits, {num_edge} edges")

    for entanglement in tqdm(["linear"]):
        for level in tqdm(range(max_level)):
            vqe = VQEForQRAO(
                hamiltonian,
                entanglement=entanglement,
                num_layer=level,
                qubit_pairs=qrac.calculate_edge_among_qubits(instance),
            )
            cost_history, best_theta_list = vqe.minimize()
            rounding = MagicRounding(m, n, shots, vqe, qrac)
            solution_counts = rounding.round(best_theta_list)
            maxcut_values = rounding.get_objective_value_counts(
                instance, solution_counts
            )

            # result of the experiment
            result = {
                "solution_counts": solution_counts,
                "maxcut_values": maxcut_values,
                "num_qubit": num_qubit,
                "num_edge": num_edge,
                "entanglement": entanglement,
                "level": level,
                "optimum_solution": instance.solve().get_objective_value(),
            }

            # save experiment result
            save_path = f"{root_path}/{m}-{n}/{entanglement}/"
            os.makedirs(save_path, exist_ok=True)
            save_file_name = f"{save_path}/level{level}.pkl"
            with open(save_file_name, "wb") as f:
                pickle.dump(result, f)

In [4]:
# search pattern
# search_pattern = {3: [8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]}
search_pattern = {3: [6]}
qrao_patterns = [(3, 1)]
# qrao_patterns = [(2, 1)]
# qrao_patterns = [(1, 1)]
MAX_LEVEL = 2
TRIAL = 1
ROUNDING_SHOTS = 1000

In [5]:
from networkx import node_link_data

for deg, num_vertices in search_pattern.items():
    for num in num_vertices:
        for m, n in qrao_patterns:
            for i in tqdm(range(TRIAL)):
                graph, instance = regular_graph(num, deg)
                root_path = f"results/regular/deg{deg}/nodes{num}/trial{i}"
                os.makedirs(root_path, exist_ok=True)
                with open(f"{root_path}/graph_data.pkl", "wb") as f:
                    pickle.dump(node_link_data(graph), f)

                run_qrao(
                    m,
                    n,
                    instance,
                    MAX_LEVEL,
                    f"results/regular/deg{deg}/nodes{num}/trial{i}",
                    ROUNDING_SHOTS,
                )

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
result = pickle.load(open("/Users/terapoon/Desktop/qrao_experiment/qulacs/results/regular/deg3/nodes6/trial0/3-1/linear/level0.pkl", "rb"))
print(result)

{'solution_counts': {'011000': 23, '100100': 14, '111111': 19, '111100': 35, '100101': 22, '100110': 4, '111101': 40, '011001': 27, '111110': 6, '011010': 9, '000001': 11, '100111': 4, '011011': 7, '101100': 26, '110100': 43, '010000': 37, '110111': 13, '110101': 44, '101101': 14, '101110': 5, '110110': 12, '001001': 8, '010001': 29, '010010': 9, '001010': 4, '010011': 9, '101111': 2, '001000': 6, '001011': 3, '000000': 4, '111011': 11, '100000': 16, '111000': 33, '100011': 7, '011100': 30, '100001': 21, '111001': 45, '111010': 9, '100010': 6, '011101': 28, '000101': 7, '110000': 51, '001100': 8, '101000': 24, '010100': 32, '110011': 17, '101001': 14, '001101': 11, '110001': 44, '110010': 11, '010101': 24, '101010': 4, '001111': 3, '101011': 8, '010110': 6, '010111': 2, '011111': 8, '000110': 1, '000100': 17, '011110': 6, '001110': 2, '000010': 3, '000011': 2}, 'maxcut_values': {4.0: 278, 6.0: 200, 5.0: 193, 3.0: 192, 7.0: 114, 0.0: 23}, 'num_qubit': 3, 'num_edge': 3, 'entanglement': '